# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Compare the results with normal holdout validation
- Apply 5-fold cross validation for regression

## Let's get started

This time, let's only include the variables that were previously selected using recursive feature elimination. We included the code to preprocess below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.datasets import load_boston

boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_target = pd.DataFrame(boston.target, columns= ["price"])
b = boston_features["B"]
logdis = np.log(boston_features["DIS"])
loglstat = np.log(boston_features["LSTAT"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))

In [2]:
X = boston_features
y = boston_target

## Train test split

Perform a train-test-split with a test set of 0.20.

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels as sm

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
lm = LinearRegression()

Fit the model and apply the model to the make test set predictions

In [6]:
lm.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

Calculate the residuals and the mean squared error

In [7]:
yhat_train = lm.predict(X_train)
yhat_test = lm.predict(X_test)
train_residuals = yhat_train - y_train
test_residuals = yhat_test - y_test
print('Train MSE: ', metrics.mean_squared_error(yhat_train, y_train))
print('Test MSE: ', metrics.mean_squared_error(yhat_test, y_test))

Train MSE:  16.417865093500676
Test MSE:  17.74412137284865


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function k-folds that splits a dataset into k evenly sized pieces.
If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [11]:
data = pd.DataFrame(boston_features)
num_observations = len(data)
fold_size = num_observations//k
leftovers = num_observations%k
folds = []
start_obs = 0
for fold_n in range(1,k+1):
    if fold_n <= leftovers:
        #Fold Size will be 1 larger to account for leftovers
        fold =  data.iloc[start_obs : start_obs+fold_size+1] 
        folds.append(fold)
        start_obs +=  fold_size + 1
    else:
        fold =  data.iloc[start_obs : start_obs+fold_size] 
        folds.append(fold)
        start_obs +=  fold_size
            
    return folds 

NameError: name 'k' is not defined

In [ ]:
from sklearn.model_selection import KFold # import KFold
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]]) # create an array
y = np.array([1, 2, 3, 4]) # Create another array
kf = KFold(n_splits=2) # Define the split - into 2 folds 
kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator
print(kf) 
KFold(n_splits=2, random_state=None, shuffle=False)

### Apply it to the Boston Housing Data

In [21]:
bos_data = pd.concat([X.reset_index(drop=True), y], axis=1)

In [22]:
bos_folds = kfolds(bos_data, 5)

ValueError: Shape of passed values is (13, 1), indices imply (1, 1)

In [18]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(bos_folds) if i!=n])
    test = bos_folds[n]
    # Fit a linear regression model
    lm.fit(train[X.columns], train[y.columns])
    #Evaluate Train and Test Errors
    y_hat_train = lm.predict(train[X.columns])
    y_hat_test = lm.predict(test[X.columns])
    train_residuals = y_hat_train - train[y.columns]
    test_residuals = y_hat_test - test[y.columns]
    train_errs.append(np.mean(train_residuals.astype(float)**2))
    test_errs.append(np.mean(test_residuals.astype(float)**2))
print(train_errs)
print(test_errs)

NameError: name 'bos_folds' is not defined

### Perform a linear regression for each fold, and calculate the training and test error

Perform linear regression on each and calculate the training and test error.

In [ ]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    # Split in train and test for the fold
    train = None
    test = None
    # Fit a linear regression model
    
    #Evaluate Train and Test Errors

# print(train_errs)
# print(test_errs)

## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [12]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

cv_5_results = cross_val_score(lm, X, y, cv=5, scoring="neg_mean_squared_error")

Next, calculate the mean of the MSE over the 5 cross-validations and compare and contrast with the result from the train-test-split case.

In [13]:
cv_5_results

array([-13.0161921 , -14.62832183, -24.81432997, -55.24107773,
       -19.022338  ])

##  Summary 

Congratulations! You now practiced your knowledge on k-fold crossvalidation!